In [1]:
import torch

In [3]:
det_box = torch.randn((1, 5, 7, 7))
gt_box = torch.randn((1, 6, 7, 7))

In [5]:
indicies = torch.arange(0, 7).repeat((7, 1))
grid = torch.cat([indicies.unsqueeze(2), indicies.T.unsqueeze(2)], 2)
grid_indicies = grid.transpose(0, 2).transpose(1, 2).unsqueeze(0)

In [8]:
det_box_area = det_box[:, 2, :, :] * det_box[:, 3, :, :]
gt_box_area = gt_box[:, 2, :, :] * gt_box[:, 3, :, :]

det_box_xycenter_rel = det_box[:, :2, :, :] + grid_indicies
det_box_xymin = det_box_xycenter_rel - (det_box[:, 2:4, :, :] / 2)
det_box_xymax = det_box_xycenter_rel + (det_box[:, 2:4, :, :] / 2)

gt_box_xycenter_rel = gt_box[:, :2, :, :] + grid_indicies
gt_box_xymin = gt_box_xycenter_rel - (gt_box[:, 2:4, :, :] / 2)
gt_box_xymax = gt_box_xycenter_rel + (gt_box[:, 2:4, :, :] / 2)

lt = torch.max(det_box_xymax.unsqueeze(1), gt_box_xymax)
rb = torch.min(det_box_xymin.unsqueeze(1), gt_box_xymin)
wh = (rb - lt).clamp(min=0)
inter = wh[:, :, 0, :, :] * wh[:, :, 1, :, :]
union = det_box_area[:, None, :, :] + gt_box_area - inter
iou = inter / union

In [13]:
iou.shape

torch.Size([1, 1, 7, 7])

In [20]:
torch.cat([iou, iou, iou], dim=1).argmax(dim=1, keepdim=True).shape

torch.Size([1, 1, 7, 7])